# STAT 628 Module2 report

**Shuo Qiang**  
**Zihan Zhou**  
**Kehui Yao**  
**Zhendong Zhou**

## Abstract

In this project, data more than 4 GB from Yelp, including customers' reviews and stores' business information are provided for manipulating. we are supposed to derive insightful information and make precise prediction. We finally choose to restrict our research within restaurants for the convenience of precision and explanation.
Two main goals for this project are:  
1. Make insightful suggestions to customers and business owners of each restaurant.  
2. Make precise prediction for customers' scoring for each restaurant based on their reviews.  

For goal 1, where we spent most of our effort on, we derived useful information from both business data and review data, made insightful suggestions and visualized output from both of them, both statistical and machine learning methods are all applied for solving this problem. Finally we built a recommendation system and published it.  
For goal 2, we first tokenized and embedded words in each review, then a self- built neural network is established for prediction. The RMSE we achieved in the end is 0.66.  

Finally we add an Anti Click Farming function and Restaurant Nearby function, merge them with our suggestion system, constructed and published an recommendation app through Shiny App.

## Goal 1

### business data

We conduct this part in 4 steps:  
1. Data preprocessing and model fitting
2. Feature importance  
3. Hypothesis testing  
4. suggestion forming  

In step 1, we mainly focused on nested information, bad-structured data, missing values and restaurants clustering. The JSON file itself has some nested dictionaries in each feature, we scanned through each feature and extracted all the nested values as new features. Then we found that there are many bad-structured data like "u'bear_and_wine'" which is nested- encoded string and "None" which is missing value with "str" type. We thus transferred all the "str" missing values into NaN in numpy and transferred all the nested-encoded strings into pure decoded strings. For the missing value part, we treat it as a new level and use Xgboost algorithm to deal with it. Finally we apply LDA to all the restaurants and got 4 clusters, we named them with "Asian", "Japan", "Fastfood" and "bars" through the key words extracted from the output of LDA.  
   
In step 2, we apply Xgboost algorithm, which is a boosting algorithm based on decision tree, to deal with missing value and get feature importance, mainly because we are fond of the "sparse aware algorithm" which is used by Xgboost to deal with missing value: it treat missing value within a feature as a new level, set it aside when learning the splitting point, then add it back to the left and right children to see the information gain, finally the children wither higher information gain will accept the missing value. The feature importance is derived based on the amount of mean loss reduction when a feature is selected. For each cluster of restaurant, we select 10 most important features as candidate, decide whether to use them based on the non-parametric hypothesis testing.  
   
In step 3, we apply Kruskal test, which is a non-parametric test that doesn't make any assumption on the distribution of the data, to see if the different levels of an exact feature does really have impact on the mean score of a restaurant. The null hypothesis of this test is that The samples come from populations with the same distribution. First we need to calculate the rank of samples in each group, then we form a so called H-statistics, which can be represented like
   $$H = (\frac{1}{s^2})[\sum_1^k \frac{R_i}{n_i} -\frac{N(N+1)^2}{4}]$$
   $$S^2 = \frac{1}{N-1}[\sum_{all}R_{ij}^2 - \frac{N(N+1)^2}{4}]$$
where k is the number of samples, $n_i$ is the size of group i, N is the total number of $n_i$, $R_{ij}$ is the rank of jth sample in ith group and $R_i$ is the sum of $R_{ij}$ in group i.
We set the set the confident coefficient 0.95 and print the features pass the test. Finally, we compared the mean score of significant features that pass the test and ranked them, as a preparation of the recommendation system. The output would be like:  
    
True    street_True, street_False  
Flase   bike_parking
    
Which means that "street" is more important than "bike" and street pass the test, "True" level for street is better than "False" level.
    
In step 4, we took a look at the output of step 3, considering if it's suitable for us to make some suggestions. Like if we have "wheelchair_accessable" pass the test, and "False" is better, we definitely can't make any suggestion that the restaurant shouldn't allow people with weelchair to enter. Finally we formed suggestions manually for all the levels of each of the features and decide which one to use based on the value in every restaurant, concatenate them togeter and write them into a csv file along with the business ID. If you enter a business ID in our file, we can then print the suggestions already formed.

### review data

When we first looked at the first 10,000 lines of the review data, three issues were found: 1. various language types, 2. emoticons and 3. numbers.  
1. In view of computation cost, we just encode each string with utf-8 and delete those '\x' part. In such way, we will remain only Latin languages.  
2. Punctuations are usually useless except emoticons. So we will consider emoticons as individual words.  
3. Numbers are sometimes very confusing. In different situation, the same number will have different meanings. So they have to be deleted.  

Then, no matter what model we are gonna use, we always want to reduce the corpus size. So the following things are conducted.
1. Expand abbreviation
    - Reduce the size of dictionary.
2. Unify adversatives
    - In such way, it is easier to create adversative phrases which will extract more information.
3. Simple spell check
    - English words which contain three or more consecutive letters are suspicious words. For example, 'noooooo'. We'll first look it up in WordNet corpus. Then check the word with only two consecutive letter exists and if not check the case with only one letter. ('noooooo' -> 'noo' -> 'no') If all these failed, a function called 'suggest' will be applied.  
4. Delete stop words  
    - Stop words are those useless words like 'the, and, is ...' which can't tell us if the restaurant is good or not.
5. Word lemmatization
    - There are many tenses in English. Our purpose is to convert verbs to their primary form
6. Adversative phrases (Only for part 1)
    - First find which sentence the adversative is in, then combine every word with the adversative. 
      
    Example:  
             Original:   I do not like the waiter.  
             Transferred: not_like not_the not_waiter
7. Find bigrams
    - The limitation of bag-of-words is that connections between words are lost. Constructing phrases can remit it . There is a function called 'phrase' in the package 'gensim'. This function first tokenize the sentence then search through all the bigrams and count each bigrams. According to its build-in bigram dictionary and the frequency calculated, the function will convert some words into a bigram form.

#### Restaurant tags
$$TF = \frac{NTD}{TTD}$$
$$IDF = log\frac{ND}{1+TD}$$
$$TfIdf = TF*IDF$$  
$NTD = $Number of times term t appears in a document  
$TTD = $Total number of terms in the document  
$TD = $Total number of documents  
$ND = $Number of documents with term t in it  
  
Words with higher TfIdf can represent the sentence better.  
So, we merged all the reviews of each restaurant and transformed them with TfIdf.
Then 5 words with the highest score were picked as the tags of the restaurant.  

#### information extraction
We constructed four key word lists for the four restaurant categories. Each of them are based on the important attributes given by XGB and Kruskal test. Given a 'business_id', we first decided which class the restaurant is in . Then for every key word in that class's list, search through all the sentences and pick out adjectives near that word. In the end, we count the frequency of those adjectives. So now we can describe how the attribute is.

## Goal 2

The goal of this model is making the sentiment analysis based on the reviews and predicting the star that a customer will give based on the review he writes. 

There are many sentiment analysis model using in NLP field in today's world. We tried many models based on the preprocessed data we have. The model and results are shown below.

![](../img/proposed_model.png)

As we can see from above, the result is pretty good for deep learning model. So maybe we can build our own deep learning model by combining the advantages of existing model to make the prediction. We know that LSTM is good at dealing sequential texts and CNN is good at extracting the local features. By combining them, the result should be better than existing ones. The final model we built is shown below.

![](../img/own_model.png)

In this model, the first layer is the embedding layer, it can transform the reviews into word vectors. I choose the embedding size be 100, and the number of unique words is 1391195 which is also the length of original one-hot vector for word in reviews. Then I add a dropout layer for higher speed and avoiding overfitting. LSTM layer is added then, there are 4 gates and we have 32 LSTM units in this layer. After that, the output will be dealt with a 1-dimentional convolution layer. There are 64 kernels in this layer, and the kernel size is 3 to work as a tri-gram model. To extract the local feature, a max-pooling layer is added behind this layer. Then we add dropout and flat is into a fully connected layer. The last layer is soft-max if we want it be a classification problem and linear if we want it be a regression problem.

The accuracy of this model in validation sets with 1 million samples is 0.715 for classification. And the mean absolute error is 0.37 for regression. The result is pretty awesome and we choose as our final model.

## Anti Click Farming

As we know, in today's world, click farming is very common, especially in scoring websites such as yelp. The owner may hire someone to fake as the real customers and give 5 stars to restaurant in order to increases their overall reputation. In other case, they may maliciously give their competitor worst score in order to attract the customer from their competitor to their own restaurant. Their are many ways to detect this behavior, but based on the data we have, we can only build a naive Anti Click Farming model to check it. The intuition is that many click farming software will only give fake star but randomly create a review. We know that the MAE of our model is 0.37, so if the average difference of predicted star and real star has a difference over 1, we have reason to suspect that there review and star can be inconsistent.

But we should notice that this system is pretty naive, we did not consider a lot of situation such as maybe the large difference is a result from the fact that many restaurants have only few reviews and scores. And we don not have the label for fake clicking, so it is hard for us to learning the feature from fake clicking. As a result, this model just a naive version and need to improve once we get more features.

## Recommendation System

The final recommendation system is built based on R-shiny. The input is the business_id of the restaurant. And the output is composed of four parts.

1. The overall distribution of customer's star score. And if the average difference of predicted star and real star diverges too much, it will warn the customers.
2. The recommendation for customers. It has two parts. The first one is the five most importance tags we extracted from the reviews. The second one is the evaluation on important business feature that extracted from the reviews. By these two parts, the customers can learn the characteristic of these restaurant and how other customers say about some important attribute of this restaurant.
3. The suggestions to restaurant. We give the structured suggestion are customized for each restaurant. The suggestions include the things that restaurant is doing great right now and things they can improve in the future.
4. The restaurant nearby. In this part, we will show a map that has the restaurant that nearby this restaurant. And if the user click on the mark on the map, it will show the average star, recommended tag, business id and the name of this restaurant. 

The app we create is on https://shuoqiang.shinyapps.io/Yelp/.

## Contribution for project
Zihan Zhou: Data pre-processing, feature extraction, restaurant clustering, attribute searching,  
Shuo Qiang: Finishing prediction model, building structure of Recommendation System.  
Kehui Yao: Business data processing, feature selection, hypothesis testing.  
Zhendong Zhou: Structured suggestion framework, parts of works in TFIDF, LDA, etc.

## Contribution for report
Zihan Zhou: Part of review section in goal 1.  
Shuo Qiang: Goal 2 and recommendation system.  
Kehui Yao: Part of business section in goal 1.  
Zhendong Zhou: Part of business section in goal 1, Abstract.  